In [157]:
!pip install -q kagglehub

import kagglehub
import os
import pandas as pd

raw_data_path = "/content/data/raw"
os.makedirs(raw_data_path, exist_ok=True)
os.environ["KAGGLEHUB_CACHE"] = raw_data_path


dataset_path = kagglehub.dataset_download("govindaramsriram/energy-consumption-dataset-linear-regression")

import os
csv_files = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".csv"):
            csv_files.append(os.path.join(root, file))

print("Archivos encontrados:")
for f in csv_files:
    print(f)



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Archivos encontrados:
/content/data/raw\datasets\govindaramsriram\energy-consumption-dataset-linear-regression\versions\1\test_energy_data.csv
/content/data/raw\datasets\govindaramsriram\energy-consumption-dataset-linear-regression\versions\1\train_energy_data.csv


In [158]:
df = pd.read_csv([f for f in csv_files if "test_energy_data.csv" in f][0])
print(f"Shape: {df.shape}")
df.head()

Shape: (100, 7)


,Building Type,Square Footage,Number of Occupants,Appliances Used,Average Temperature,Day of Week,Energy Consumption
0,Residential,24563,15,4,28.52,Weekday,2865.57
1,Commercial,27583,56,23,23.07,Weekend,4283.80
2,Commercial,45313,4,44,33.56,Weekday,5067.83
3,Residential,41625,84,17,27.39,Weekend,4624.30
4,Residential,36720,58,47,17.08,Weekday,4820.59


In [159]:
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

print("Columnas categóricas detectadas:")
print(cat_cols)


Columnas categóricas detectadas:
['Building Type', 'Day of Week']


In [160]:
for c in ['Building Type', 'Day of Week']:
    df[c] = df[c].astype(str).str.strip().str.title()

print("Building Type:", sorted(df['Building Type'].unique()))
print("Day of Week:", sorted(df['Day of Week'].unique()))

Building Type: ['Commercial', 'Industrial', 'Residential']
Day of Week: ['Weekday', 'Weekend']


In [161]:
cat_cols = ['Building Type', 'Day of Week']

df_encoded = pd.get_dummies(
    df,
    columns=cat_cols,
    drop_first=False,
    dummy_na=False,
    dtype=int
)

print("Shape después de One-Hot:", df_encoded.shape)
[d for d in df_encoded.columns if d.startswith('Building Type_') or d.startswith('Day of Week_')]

Shape después de One-Hot: (100, 10)


['Building Type_Commercial',
 'Building Type_Industrial',
 'Building Type_Residential',
 'Day of Week_Weekday',
 'Day of Week_Weekend']

In [162]:
print("Shape:", df_encoded.shape)

cols = df_encoded.columns.tolist()
print("Total columnas:", len(cols))
cols
df_encoded.head(10)


Shape: (100, 10)
Total columnas: 10


,Square Footage,Number of Occupants,Appliances Used,Average Temperature,Energy Consumption,Building Type_Commercial,Building Type_Industrial,Building Type_Residential,Day of Week_Weekday,Day of Week_Weekend
0,24563,15,4,28.52,2865.57,0,0,1,1,0
1,27583,56,23,23.07,4283.80,1,0,0,0,1
2,45313,4,44,33.56,5067.83,1,0,0,1,0
3,41625,84,17,27.39,4624.30,0,0,1,0,1
4,36720,58,47,17.08,4820.59,0,0,1,1,0
5,31207,47,28,22.82,5026.23,0,1,0,1,0
6,39227,18,44,23.36,4404.56,0,0,1,0,1
7,7814,21,19,27.27,2394.37,0,0,1,1,0
8,20482,24,16,23.00,3969.09,0,1,0,0,1
9,21030,90,35,12.96,5136.69,0,1,0,1,0


In [163]:
output_path = "../data/processed/energy_data_processed_test.csv"
df_encoded.to_csv(output_path, index=False)
print(f"CSV guardado en: {output_path}")

CSV guardado en: ../data/processed/energy_data_processed_test.csv
